# Apprentissage par renforcement

Le but de ce tutoriel est de vous montrer une implémentation simple de deux algorithmes très bien connus dans le domaine : le *Deep Q-Learning* et le *Proximal Policy Optimization*. Il y a plusieurs `TODO` à des endroits où vous devrez ajouter du code. La solution se trouve dans le fichier `solution_tutoriel_apprentissage_renforcement.ipynb`.

In [ ]:
%%capture
!uv pip install -r requirements.txt

import torch
import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
import numpy as np
from copy import deepcopy
from collections import namedtuple, deque
import random
import lightning as L
from torch.distributions import MultivariateNormal
from tqdm import tqdm, trange
from utils import get_device, verify_shape

# Essayez les jeux

Vous pouvez essayer un jeu, vous avez le choix entre "Car", "Mountain" et "Lunar". Vous pouvez jouer avec les touches "a", "s", "d" et "w".

In [ ]:
!python play.py --game="Mountain"

# Deep Q-Learning

Cette section du laboratoire est fortement inspirée de ces deux tutoriels suivants : 
1. https://huggingface.co/learn/deep-rl-course/en/unit3/deep-q-algorithm

2. https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

<img src="./images/deep-q-learning.jpg" width="70%" class="center"/>

[source](https://huggingface.co/learn/deep-rl-course/en/unit3/deep-q-algorithm)

Commençons par implémenter le *replay buffer*. Le buffer a une taille maximale, donc lorsque le buffer est plein, on efface les données les plus anciennes.

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory:
    def __init__(self, max_capacity):
        self.memory = [] # Choisissez la structure de données que vous préférez
        self.max_capacity = max_capacity

    def push(self, state, action, next_state, reward):
        transition = Transition(state, action, next_state, reward)
        # TODO: Ajoutez les données à la mémoire, en s'assurant que le buffer n'est pas plein.
        ...

    def sample(self, batch_size):
        # TODO : piger aléatoirement une batch de taille batch_size
        ...

    def __len__(self):
        return len(self.memory)

On va maintenant implémenter le réseau de neurones. Ils suggèrent un réseau de neurones avec deux couches cachées de 128 neurones, donc c'est ce qu'on utilise ici.

In [ ]:
class DQN(torch.nn.Module):
    def __init__(self, n_obs, n_actions, hidden_size=128):
        super().__init__()
        self.n_obs = n_obs
        self.n_actions = n_actions
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(self.n_obs, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, self.n_actions)
        )

    def forward(self, input):
        return self.layers(input)

On implémente l'étape d'entraînement de l'algorithme présenté plus haut.

In [ ]:
def update_model(model, frozen_model, batch_size, memory, gamma, optimizer):
    model.train()
    # Allez chercher une batch dans la mémoire
    batch = memory.sample(batch_size)
    batch = Transition(*zip(*batch))
    all_states = torch.tensor(np.stack(batch.state), device=get_device())
    all_next_states = batch.next_state

    # TODO : Créez un tenseur qui vérifie que next_state est un état valide et non un état terminal. Il doit contenir
    # True si next_state est valide.
    mask = torch.tensor(...
                        , dtype=torch.bool, device=get_device())

    # TODO : Créez un tenseur qui contient tous les next_state valides .
    non_final_next_states = torch.tensor(np.stack(...
                                                 ), device=get_device())

    rewards = torch.tensor(batch.reward, device=get_device())
    actions = torch.tensor(batch.action, device=get_device()).unsqueeze(1)
    targets = rewards
    with torch.no_grad():
        targets[mask] += gamma * frozen_model(non_final_next_states).max(1).values
    ys = model(all_states).gather(1, actions).reshape(-1,)

    # TODO : Calculez la moyenne de la distance au carré entre targets et ys.
    loss = ...
    
    optimizer.zero_grad()
    loss.backward()

    # On clip le gradient pour assurer la stabilité
    torch.nn.utils.clip_grad_value_(model.parameters(), 100)
    optimizer.step()

    model.eval()

On implémente maintenant la boucle d'entraînement du modèle.

In [ ]:
def train_model(env, config):
    obs, info = env.reset()
    size_observation_space = len(obs)
    n_actions = env.action_space.n

    epsilon = config['epsilon_start']

    # Initialize replay memory D to capacity N
    memory = ReplayMemory(max_capacity=config['max_capacity'])
    # Initialize action-value function Q with random weights theta
    Q_model = DQN(size_observation_space, n_actions, config['hidden_layer_size'])

    # Initialize target action-value function \hat{Q} with weights theta^-1 = theta
    Q_hat_model = deepcopy(Q_model)

    Q_model.eval()
    Q_hat_model.eval()
    Q_model.to(get_device())
    Q_hat_model.to(get_device())

    optimizer = torch.optim.Adam(Q_model.parameters(), lr=config['lr'])

    global_step_count = 0
    for episode_num in range(config['num_eval_episodes']):
        obs, info = env.reset()
        episode_reward = 0
        step_count = 0

        # On crée notre distribution pour le compromis exploration-exploitation
        epsilon_greedy = torch.distributions.bernoulli.Bernoulli(torch.tensor([epsilon]))
        for n_step in range(env.spec.max_episode_steps):
            # Epsilon decay
            if bool(epsilon_greedy.sample().item()):
                action = env.action_space.sample()  # Pige aléatoire
            else:
                with torch.no_grad():
                    action = torch.argmax(Q_model(torch.tensor(obs, device=get_device()))).item()

            new_obs, reward, terminated, truncated, info = env.step(action)

            episode_terminated = terminated or truncated
            episode_reward += reward
            step_count += 1
            global_step_count += 1

            # On ajoute les données dans la mémoire. Si l'état est terminal, on ajoute None au lieu de l'état
            if episode_terminated:
                memory.push(obs, action, None, reward)
            else:
                memory.push(obs, action, new_obs, reward)
            obs = new_obs

            # Dès qu'on a assez de données, on se met à entraîner le modèle
            if len(memory) >= config['batch_size']:
                update_model(Q_model, Q_hat_model, config['batch_size'], memory, config['gamma'], optimizer)

            # Après plusieurs étapes, on met à jour notre modèle gelé
            if global_step_count % config['n_step_update'] == 0:
                Q_hat_model = deepcopy(Q_model)

            if episode_terminated:
                break

        print(f"Episode {episode_num + 1}: {step_count} steps, reward = {episode_reward}")
        epsilon *= pow(config['epsilon_end']/config['epsilon_start'], 1/num_eval_episodes)

Voici le code qui permet de rouler l'entièreté de l'algorithme. Vous pouvez jouer avec les hyperparamètres, mais ceux que j'ai utilisés fonctionnent bien.

In [ ]:
L.pytorch.seed_everything(42, workers=True)

num_eval_episodes = 450
env_name = "MountainCar-v0"

# Create environment with recording capabilities
env = gym.make(env_name, render_mode="rgb_array")  # rgb_array needed for video recording

# Add video recording for every episode
env = RecordVideo(
    env,
    video_folder=env_name + "_agent",    # Folder to save videos
    name_prefix="eval",               # Prefix for video filenames
    episode_trigger=lambda x: x % 10 == 0    # Record every episode
)

# Add episode statistics tracking
env = RecordEpisodeStatistics(env, buffer_length=num_eval_episodes)

print(f"Starting evaluation for {num_eval_episodes} episodes...")

config = {
    "num_eval_episodes":num_eval_episodes,
    "n_step_update": 10,
    "max_capacity" : 10000,
    "batch_size" : 128,
    "gamma" : 0.99,
    "lr" : 3e-4,
    "epsilon_start" : 0.9,
    "epsilon_end" : 0.01,
    "hidden_layer_size":128,
}

train_model(env, config)

env.close()

# Calculate some useful metrics
avg_reward = np.mean(env.return_queue)
avg_length = np.mean(env.length_queue)
std_reward = np.std(env.return_queue)

print(f'\nAverage reward: {avg_reward:.2f} ± {std_reward:.2f}')
print(f'Average episode length: {avg_length:.1f} steps')
print(f'Success rate: {sum(1 for r in env.return_queue if r > -200) / len(env.return_queue):.1%}')

# Proximal Policy Optimization

Cette section du laboratoire est fortement inspirée de cette implémentation : 
1. https://github.com/ericyangyu/PPO-for-Beginners

<img src="./images/ppo_algo.png" width="70%" class="center"/>

Les deux réseaux de neurones ont la même architecture, donc j'ai créé seulement une classe pour faire les deux réseaux.

In [ ]:
class PPONetwork(torch.nn.Module):
    def __init__(self, n_obs, out_dim, hidden_size=64, std_val=0.5):
        super().__init__()
        self.n_obs = n_obs
        self.out_dim = out_dim
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(self.n_obs, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, self.out_dim)
        )
        self.cov_mat = torch.diag(torch.full(size=(self.out_dim,), fill_value=std_val))
        self.tanh = torch.nn.Tanh()

    def forward(self, input):
        return self.layers(input)

    def get_action(self, input):
        moyenne = self(input)
        dist = MultivariateNormal(moyenne, self.cov_mat.to(get_device()))

        # TODO : On pige une action dans la distribution
        action = ...

        # TODO : On calcule la log_probabilité de l'action en fonction de la distribution
        log_prob = ...

        return action.cpu().numpy(), log_prob.detach().cpu().item()

    def evaluate(self, input, actions):
        dist = MultivariateNormal(self(input), self.cov_mat.to(get_device()))
        log_probs = dist.log_prob(actions)
        return log_probs

On va maintenant estimer les "avantages". Pour ce faire, on va calculer la somme actualisée avec le paramètre gamma pour chaque épisodes. 
<img src="./images/advantages.png" width="50%" class="center"/>

Plus exactement, pour chaque épisode, on va chercher à calculer :
$\sum_{t=0}^T \gamma^t r_t$

In [ ]:
def calculate_estimated_advantages(batch_rewards, gamma):
    batch_rtgs = []
    # On calcule les rewards diminués pour chaque épisode
    for ep_rewards in reversed(batch_rewards):

        discounted_reward = 0  # Somme des rewards diminués

        # On itère sur tous les rewards de l'épisode
        for rew in reversed(ep_rewards):
            # TODO : On additionne le reward actual avec les rewards obtenus précédemment multipliés par gamma.
            discounted_reward = ...
            batch_rtgs.insert(0, discounted_reward)

    batch_rtgs = torch.tensor(batch_rtgs, dtype=torch.float)
    return batch_rtgs

On fait jouer plein d'agents différents pour obtenir les données sur lesquelles on va s'entraîner.

In [ ]:
def rollout(env, n_actors, iter_number, actor, critic):
    batch_obs = []
    batch_actions = []
    batch_log_probs = []
    batch_rewards = []

    for n_actor in trange(n_actors, desc="Actors"):
        obs, info = env.reset(seed=(iter_number+1)*(n_actor+1))
        ep_rewards = []
        for n_step in range(env.spec.max_episode_steps):
            action, log_prob = actor.get_action(torch.tensor(obs, device=get_device()))
            new_obs, reward, terminated, truncated, info = env.step(action)

            ep_rewards.append(reward)
            batch_obs.append(obs)
            batch_actions.append(action)
            batch_log_probs.append(log_prob)

            obs = new_obs

            episode_terminated = terminated or truncated

            if episode_terminated:
                break
        batch_rewards.append(ep_rewards)

    batch_obs = torch.tensor(np.array(batch_obs), dtype=torch.float)
    batch_actions = torch.tensor(np.array(batch_actions), dtype=torch.float)
    batch_log_probs = torch.tensor(np.array(batch_log_probs), dtype=torch.float)
    
    return batch_obs, batch_actions, batch_log_probs, batch_rewards

On met maintenant à jour les réseaux de neurones. On calcule la perte avec l'équation suivante. Le ratio $r_t$ est calculé pour vous.

<img src="./images/l_clip.png" width="50%" class="center"/>

In [ ]:
def update_models(dataloader, actor, critic, optimizer_actor, optimizer_critic, epsilon, n_epochs):
        actor.train()
        critic.train()
        device = get_device()
        loss_fn = torch.nn.MSELoss()
        for _ in trange(n_epochs, desc="Epochs"):
            for batch in dataloader:
                b_obs = batch[0].to(device)
                b_acts = batch[1].to(device)
                b_log_probs = batch[2].to(device)
                b_rtgs = batch[3].to(device)
                b_ak = batch[4].to(device)
                V = critic(b_obs).squeeze()
                curr_log_probs = actor.evaluate(b_obs, b_acts)
                ratios = torch.exp(curr_log_probs - b_log_probs)

                # TODO : On calcule la première partie de la perte 
                surr1 = ...
                # TODO : On calcule la deuxième partie de la perte, avec la fonction torch.clamp
                surr2 = ...

                # On calcule la perte pour l'acteur
                actor_loss = (-torch.min(surr1, surr2)).mean()
                optimizer_actor.zero_grad()
                actor_loss.backward()
                optimizer_actor.step()

                # On calcule la perte pour le critique
                V, b_rtgs = verify_shape(V, b_rtgs)
                critic_loss = loss_fn(V, b_rtgs)

                optimizer_critic.zero_grad()
                critic_loss.backward()
                optimizer_critic.step()
        actor.eval()
        critic.eval()

Voici la boucle d'entraînement. On fait notre "rollout", où on crée les données sur lesquelles on va s'entraîner. Ensuite, on estimer l'avantage et on entraîne le modèle, avec les méthodes qu'on a implémentés plus haut.

In [ ]:
def train_model(env, config):
    obs, info = env.reset()
    size_observation_space = len(obs)
    n_actions = env.action_space.shape[0]

    actor = PPONetwork(size_observation_space, n_actions, config['hidden_layer_size'])
    critic = PPONetwork(size_observation_space, 1, config['hidden_layer_size'])

    optimizer_actor = torch.optim.Adam(actor.parameters(), lr=config['lr'], eps=1e-5)
    optimizer_critic = torch.optim.Adam(critic.parameters(), lr=config['lr'], eps=1e-5)

    actor.to(get_device())
    actor.eval()
    critic.to(get_device())
    critic.eval()

    for i in range(config['num_eval_episodes']):
        batch_obs, batch_actions, batch_log_probs, batch_rewards = rollout(env, config['n_actors'], i, actor, critic)

        final_rewards = np.array([sum(ep_rewards) for ep_rewards in batch_rewards])
        print(f"Average reward : {final_rewards.mean():.2f}   Max final reward : {final_rewards.max():.2f}")
        print(f"Pourcentage de réussite après {i} itérations: {100*np.count_nonzero(final_rewards>=200)/final_rewards.shape[0]:.2f}%")

        batch_rtgs = calculate_estimated_advantages(batch_rewards, config['gamma'])
        
        V = critic(batch_obs.to(get_device())).squeeze()
        A_k = batch_rtgs.to(get_device()) - V.detach()
        A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10) # Normalization trick
        
        dataset = torch.utils.data.TensorDataset(batch_obs, batch_actions, batch_log_probs, batch_rtgs, A_k)
        dataloader = torch.utils.data.DataLoader(dataset,batch_size=config['batch_size'], shuffle=True)
        update_models(dataloader, actor, critic, optimizer_actor, optimizer_critic, config['epsilon'], config['n_epochs'])

Voici le code qui permet de rouler l'entièreté de l'algorithme. Vous pouvez jouer avec les hyperparamètres, mais ceux que j'ai utilisés fonctionnent bien.

In [ ]:
L.pytorch.seed_everything(42, workers=True)

num_eval_episodes = 10
env_name = "LunarLanderContinuous-v3"

# Create environment with recording capabilities
env = gym.make(env_name, render_mode="rgb_array")  # rgb_array needed for video recording

# Add video recording for every episode
env = RecordVideo(
    env,
    video_folder=env_name + "_agent",    # Folder to save videos
    name_prefix="eval",               # Prefix for video filenames
    episode_trigger=lambda x: x % 50 == 0    # Record every 50 episode
)

# Add episode statistics tracking
env = RecordEpisodeStatistics(env, buffer_length=num_eval_episodes)

print(f"Starting evaluation for {num_eval_episodes} iterations...")

config = {
    "num_eval_episodes":num_eval_episodes,
    'n_actors':20,
    'n_epochs':5,
    "batch_size" : 64,
    "gamma" : 0.99,
    "lr" : 3e-4,
    "epsilon" : 0.2,
    "hidden_layer_size":64,
}

train_model(env, config)

env.close()

# Calculate some useful metrics
avg_reward = np.mean(env.return_queue)
avg_length = np.mean(env.length_queue)
std_reward = np.std(env.return_queue)

print(f'\nAverage reward: {avg_reward:.2f} ± {std_reward:.2f}')
print(f'Average episode length: {avg_length:.1f} steps')
print(f'Success rate: {sum(1 for r in env.return_queue if r > 200) / len(env.return_queue):.1%}')